#### Kalman Filter
This implementation was made for position and velocity but it can be extended and generalized.

Initial matrix $\vec{x} = \begin{pmatrix} p \\ v \end{pmatrix}$

In [16]:
using Statistics
using LinearAlgebra

In [17]:
x = [p,v]
P = Symmetric([cov(p,p) cov(p,v); cov(p,v) cov(v,v)])

LoadError: UndefVarError: p not defined

Now we put the relation between p and v in $F_t$ matrix to compute $t_k$ from $t_{k-1}$

t_change is $\Delta t$

In [23]:
t_change
F = [1 t_change; 0 1]
x_k = F*x

#x_k is our prediction matrix 

P_k = F*P*transpose(F)

#P_k is next cor matrix

LoadError: UndefVarError: t_change not defined

#### External influence 

If there was an accelaration included, we can include that in our mat

$B_k$ is control matrix and $\vec{u_k}$ is control vector

In [ ]:
# so our new x_k will become 
# a is the acceleration 

B = [(t_change^2)/2;t_change] #be the matrix for external infl
u = a #be the acceleration mat

x_k = x_k + B*u


#### External Uncertainty 
Some external forces can't be tracked so, we always have some uncertainty with $x_k$

Let $Q_k$ represent the uncertainty cov matrix 

In [ ]:
P_k = P_k + Q_k

#### Refining estimate with measurements
To model the sensors, we create $H_k$ with $\vec{p}$ and $\vec{v}$

Every intital vector to be multiplied with H to establish a relation 
between sensors and velocity, position.

$\vec{\mu}_{expected}$ = $H_k\hat{x}_k$

$\sum_{expected}$ = $H_kP_kH^T_k$

In [ ]:
H
u_0 = H*x_k
E_0 = H*P_k*transpose(H)


But there can be a mismatch in actual readings vs the deduced reading
So, there will be two estimates $H_k\hat{x}_k$ and $\vec{z}_k$

$(\mu_0 , \sum_0 ) = (H_k\hat{x}_k, H_k+_kH^T_k)$


$(\mu_1, \sum_1) = (\vec{z}_k, R_k)$

Now, these two will have two different guassian distribution.
We take the area which lies in both es1 and es2 (overlapped region)

Product of two Gaussians with no correlation has following values
$ \mu^{'} = \mu_0 + \frac{\sigma^2_0(\mu_1 - \mu_0)}{\sigma^2_0 + \sigma^2_1}$

$\sigma^{'2}_0 = \sigma^2_0 - \frac{\sigma^4_0}{\sigma^2_0 + \sigma^2_1}$

Let K = $\frac{\sigma^2_0}{\sigma^2_0 + \sigma^2_1}$

$\begin{equation} 
\begin{aligned} 
\mathbf{H}_k \color{royalblue}{\mathbf{\hat{x}}_k’} &= \color{fuchsia}{\mathbf{H}_k \mathbf{\hat{x}}_k} & + & \color{purple}{\mathbf{K}} ( \color{yellowgreen}{\vec{\mathbf{z}_k}} – \color{fuchsia}{\mathbf{H}_k \mathbf{\hat{x}}_k} ) \\ 
\mathbf{H}_k \color{royalblue}{\mathbf{P}_k’} \mathbf{H}_k^T &= \color{deeppink}{\mathbf{H}_k \mathbf{P}_k \mathbf{H}_k^T} & – & \color{purple}{\mathbf{K}} \color{deeppink}{\mathbf{H}_k \mathbf{P}_k \mathbf{H}_k^T} 
\end{aligned} \label {kalunsimplified} 
\end{equation}$

and Kalman Gain is:

$\begin{equation} \label{eq:kalgainunsimplified} 
\color{purple}{\mathbf{K}} = \color{deeppink}{\mathbf{H}_k \mathbf{P}_k \mathbf{H}_k^T} ( \color{deeppink}{\mathbf{H}_k \mathbf{P}_k \mathbf{H}_k^T} + \color{mediumaquamarine}{\mathbf{R}_k})^{-1} 
\end{equation}$

Now, we can remove $H_k$ from every term and we get

$\begin{equation} 
\begin{split} 
\color{royalblue}{\mathbf{\hat{x}}_k’} &= \color{fuchsia}{\mathbf{\hat{x}}_k} & + & \color{purple}{\mathbf{K}’} ( \color{yellowgreen}{\vec{\mathbf{z}_k}} – \color{fuchsia}{\mathbf{H}_k \mathbf{\hat{x}}_k} ) \\ 
\color{royalblue}{\mathbf{P}_k’} &= \color{deeppink}{\mathbf{P}_k} & – & \color{purple}{\mathbf{K}’} \color{deeppink}{\mathbf{H}_k \mathbf{P}_k} 
\end{split} 
\label{kalupdatefull} 
    \end{equation}$

<img src = "https://www.bzarg.com/wp-content/uploads/2015/08/kalflow.png">

In [ ]:
#now we convert previous values in the form of K

K = P_k*transpose(H_k)*inv(H_k*P_k*transpose(H_k) + R_k)  

x_final = *x_k + K(z_k - H*x_k)
P_final = P_k - K*H_k*P_k
